In [2]:
## FUNCTIONS
def getRawHTML(link):
    
    r = get(link, headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    
    return soup

def retrieveServiceData(data):
    
    all = data.find_all('table', {"class": "typetable"})
    
    serviceArr = []
    for item in all[0].find_all('tr'):
        serviceObj = {}
        
        try:
            service = item.find_all('td')[0].find('a', {"class": "link"}).text
            serviceObj['service'] = service
        except:
            service = 'NA'
        
        try:
            desc = item.find_all('td')[1].text.replace("\n", "")
            serviceObj['description'] = desc
        except:
            desc = 'NA'
        
        try:
            postStr = item.find('a', {"class": 'link'})['href']
            link = base_url + postStr
            serviceObj['link'] = link
        except:
            link = 'NA'
    
        if serviceObj:
            serviceArr.append(serviceObj)
    
    fileDir = dirPath +'/service'
    if not path.exists(fileDir):
        mkdir(fileDir)
        
    with open(path.join(fileDir, '') + 'service.json', 'w+') as outfile:
        dump(serviceArr, outfile)
        
    return serviceArr

def retrieveOperationData(data, index):
    
    all = data.find_all('table', {"class": "typetable"})

    operationArr = []
    for item in all[0].find_all('tr'):
        operationOj = {}
        
        try:
            ops = item.find_all('td')[0].find('a', {"class": "link"}).text
            operationOj['operation'] = ops
            if item.find_all('td')[0].find('span').text:
                operationOj['isDeprecated'] = True if item.find_all('td')[0].find('span') else False
        except:
            ops = 'NA'
        
        try:
            desc = item.find_all('td')[1].text.replace("\n", "").replace("\u00a0", "")
            operationOj['description'] = desc
        except:
            desc = 'NA'
        
        try:
            postStr = item.find('a', {"class": 'link'})['href']
            link = base_url + serviceArr[index]['service'] + '/v38.2/' + postStr
            link = link.strip()
            operationOj['link'] = link
        except:
            link = 'NA'
    
        if operationOj:
            operationOj['service'] = serviceArr[index]['service']
            operationArr.append(operationOj)
    
    fileDir = dirPath +'/operation'
    if not path.exists(fileDir):
        mkdir(fileDir)
        
    individual_ops_target_dir = fileDir + '/'+ serviceArr[index]['service'] +'/'
    if not path.exists(individual_ops_target_dir):
        mkdir(individual_ops_target_dir)
    
    with open(path.join(individual_ops_target_dir, '') + serviceArr[index]['service'] + '.json', 'w+') as outfile:
        dump(operationArr, outfile)
    
    return operationArr

def retrieve(hrefName, data):
    fieldsArr = []
    for i in data:
        for j in i.find_all('td'):
            if j.find('a', {"name": hrefName}):
                fieldsData = (j.find('table', {"class": 'typetable'})).find_all('tr')
                for item in fieldsData:
                    if len(item.find_all('td')) > 2:
                        fieldsObj = {}
                        try:
                            fieldsObj['parameterName'] = item.find_all('td')[0].text.replace("\n", "").replace("\xa0", "")
                        except:
                            fieldsObj['parameterName'] = 'NA'

                        try:
                            fieldsObj['type'] = item.find_all('td')[1].text.replace("\n", "").replace("\xa0", "")
                            if ((item.find_all('td')[1]).find('a')):
                                if ((item.find_all('td')[1]).find('a')).get('href'): 
                                    fieldsObj[item.find_all('td')[1].text.replace("\n", "").replace("\xa0", "")] = retrieve(((item.find_all('td')[1]).find('a')).get('href')[1:], data)
                        except:
                            fieldsObj['type'] = 'NA'
                            
                        try:
                            fieldsObj['cardinality'] = item.find_all('td')[2].text.replace("\n", "").replace("\xa0", "")
                        except:
                            fieldsObj['cardinality'] = 'NA'
    
                        try:
                            fieldsObj['description'] = item.find_all('td')[3].text.replace("\n", "").replace("\xa0", "")
                        except:
                            fieldsObj['description'] = 'NA'
                            
                        try:
                            if (item.find_all('td')[4]).find('table'):
                                
                                validation_data = (item.find_all('td')[4]).find_all('table')
                                validateArr = []
                                for val in validation_data[0].find_all('tr'):
                                    validateObj = {}
                                    if len(val.find_all('td')):
                                        try:
                                            validation = val.find_all('td')[0].text
                                            validateObj['validation'] = validation
                                        except:
                                            validation = 'NA'

                                        try:
                                            description = val.find_all('td')[1].text
                                            validateObj['description'] = description
                                        except:
                                            validation = 'NA'

                                        if validateObj:
                                            validateArr.append(validateObj)
                                fieldsObj['validation'] = validateArr
                        except:
                            fieldsObj['validation'] = 'NA'
                            
                        if fieldsObj:
                            fieldsArr.append(fieldsObj)
        
    return fieldsArr

def retrieveFieldData(link, service, operation):
    
    rawFieldData = getRawHTML(link)    
    ulList = rawFieldData.find_all('ul')[2]
    requesthreflink = ((ulList.find('li')).find('a'))['href'][1:]
    data = rawFieldData.find_all('table')[0].find_all('tr')
    xyz = retrieve(requesthreflink, data)

    fileDir = dirPath +'/operation/' + service + '/fields'
    if not path.exists(fileDir):
        mkdir(fileDir)
    
    with open(path.join(fileDir, '') + operation + ".json", "w+") as outfile:
        dump(xyz, outfile)
        
    return True


from requests import get
from json import dump
from os import getcwd
from os import path
from os import mkdir
from tqdm import tqdm
from bs4 import BeautifulSoup
from multiprocessing import Pool, Process


dirPath = getcwd()

base_url = "https://community.workday.com/sites/default/files/file-hosting/productionapi/"

## Service Extraction
rawServiceData = getRawHTML("https://community.workday.com/sites/default/files/file-hosting/productionapi/index.html")
serviceArr = retrieveServiceData(rawServiceData)
pool=Pool(processes=5)
process = []
## Operations Extraction
for index in range(len(serviceArr)):
    rawOperationData = getRawHTML(serviceArr[index]['link'].strip())
    operationArr = retrieveOperationData(rawOperationData, index)
    


 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True
 DEPRECATED
True


In [1]:
from os import getcwd, path, mkdir, cpu_count

n = cpu_count()
print(n)

4
